# Experimenting with LlaMA2 for deliberAIde

## Download the Model via Hugging Face API

The following code takes advantage of Hugging Face API and Google Colab's free-tier GPU. With this tier, we are able to use Llama 2 7B version. Higher versions (i.e. more parameters) requires additional VRAM, and a higher (no longer free) tier.

First, install necessary packages and log in using hugging face.

In [ ]:
!pip install huggingface_hub
!pip install -q transformers einops accelerate langchain bitsandbytes

in command line, use `huggingface-cli login` to authenticate with Hugging Face API key.

After that, you can run code below to download the model. 

In [ ]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = transformers.pipeline(
    "text-generation", 
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=1000,
    eos_token_id=tokenizer.eos_token_id
)

In [ ]:
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})

## Prompt-Template

Here is a simple prompt template. I have included the specific template just for topics for now. We can extend this to the other prompts. 

In [ ]:
from langchain import PromptTemplate,  LLMChain

template = """
            You are an assistant for group discussions, specializing in keeping track of and documenting the discussion,/
            that is, the topics discussed, the viewpoints/positions on each topic, and the arguments/explanations given in support of each viewpoint./
            List the topics in an array according to the following example: [[" ", "topic1"],[" ","topic2"]]
            Identify the one or several main topics discussed in the discussion transcript, delimited with triple backticks. If there are multiple identified topics, but they all center around the same main topic, only record the main topic. Don't record sub-topics.

            Return the topics as a list in json format.
            Review transcript: '''{transcript}'''
           """

prompt = PromptTemplate(template=template, input_variables=["transcript"])

topics_chain = LLMChain(prompt=prompt, llm=llm)

## Transcripts

Easy access to test transcripts.

In [ ]:
# Social Media
transcript1 = 'Participant 1: "The role of social media in political campaigns is a subject that has gained significant attention in recent years.\
    It has become a powerful tool for politicians to engage with voters and spread their message. \
    However, there are concerns about the spread of misinformation and the manipulation of public opinion through targeted ads."\
    Participant 2: "I agree that social media provides a platform for political candidates to connect with a wider audience and mobilize support.\
    However, the lack of regulation and transparency in political advertising on these platforms is a major issue that needs to be addressed."\
    Participant 3: "I believe that social media has democratized political discourse and allowed marginalized voices to be heard. \
    It provides a platform for grassroots movements and enables citizens to participate in political discussions like never before.\
    We should focus on educating users about media literacy and critical thinking to combat misinformation." \
    Participant 4: "While social media has its benefits, the algorithms used by these platforms tend to create echo chambers and reinforce existing biases.\
    We need better regulation to ensure that diverse viewpoints are represented and to prevent the manipulation of public opinion through targeted content."'

In [ ]:
# Cyber-Security
transcript2 = 'Speaker 1: "With the rising threat of cyber-attacks, our data is at risk.\
    We need to act now, and I believe that multi-factor authentication can be a part of the solution.\
    "Speaker 2: "Well, I agree about the threat, but multi-factor authentication is a hassle. \
    We have to consider the inconvenience it could cause."Speaker 3: "Inconvenience is a small price to pay for security. \
    Besides, once it becomes routine, it won\'t feel inconvenient.\
    "Speaker 4: "Yes, but what about older folks who struggle with technology? \
    It\'s important to keep our systems user-friendly.\
    "Speaker 5: "I think the key is balance. We need a solution that ensures both security and ease of use.\
    "Speaker 6: "Implementing multi-factor authentication isn\'t enough. \
    We need to raise public awareness about cybersecurity risks and educate people about safe practices."Speaker 7: "I agree. \
    Let\'s not forget the importance of cybersecurity policies and regular system audits too."'



In [ ]:
# Marijuana
transcript3 = 'P1: "The debate around the legalization of recreational marijuana has been a hot topic recently.\
I personally think it\'s high time we embraced this change.\
It could be a new source of tax revenue and reduce crime related to illegal drug trafficking."\
P2: "I agree with the economic benefits, P1, but aren\'t we forgetting about the public health implications? \
Increased accessibility might lead to misuse and addiction. I\'m worried about our youth." \
P3: "The public health concern is valid, P2. But, alcohol and tobacco are legal and potentially harmful too, isn\'t it a matter of personal responsibility and proper regulation?" \
P4: "P3 makes a good point. Regulation is the key. Maybe we can use some of the tax revenue for public health campaigns and addiction treatment services?" P5: "And let\
\'s not forget, legalization also means we can have quality control standards for marijuana, which isn\'t possible now. \
It might actually make it safer." P6: "True, but there\'s also the image issue. \
We don\'t want to be seen as a city promoting drug use, do we?" P7: "It\'s a tricky decision indeed. \
Maybe we should hold a citywide referendum and let our citizens decide? In the meantime, we should gather more data and engage experts in public health, \
law enforcement and economic policy to ensure we have all the facts."'

In [ ]:
# Youth Violence
transcript4 = 'Speaker 1: "We\'ve seen a rise in youth violence in our community, and I\'m worried. \
Maybe some sort of mentoring program could make a difference?"\
Speaker 2: "Mentoring? Really? In my opinion, we need more police presence. That\'s the only language these kids understand." \
Speaker 3: "I wouldn\'t discount the value of a mentoring program so quickly. \
The problem isn\'t just law enforcement, but also social issues. Addressing those might help." \
Speaker 4: "If you ask me, this is a problem that begins at home. Many of these kids lack a good family structure. \
We need programs that support families too." Speaker 5: "All these are good points, but what about the role of schools? \
They\'re underfunded and struggling to offer good education and extracurricular activities." \
Speaker 6: "Yes, and we also have to remember the role of peer pressure in youth violence. \
We need programs that teach our kids about healthy relationships and choices."'

## Running

Now we are ready to pass in one of the transcripts and see how it does on the topic assessment.

In [ ]:
print(topics_chain.run(transcript1)) # Social Media Transcript

# Extending beyond Topics

## Viewpoints and Arguments


I have developed some general frameworks for when we want to see the accuracy and stability of viewpoints (`viewpoints_chain`) and arguments (`arguments_chain`).

In [ ]:
template = """
        You are an assistant for group discussions, specialized on keeping track and documenting the discussion,
        that is, the topics discussed, the viewpoints/positions on each topic and the arguments/explanations given in support of each viewpoint.

        For each main topic in '{topics}', analyse the corresponding excerpt from the below discussion transcript, deliminted by triple backticks.
        Your task is to identify all the viewpoints expressed on the topic.
        For each topic, create a list of viewpoints in the following json format: [["topic1","viewpoint1"],["topic1","viewpoint2"]].

        Proceed according to the following steps:

        Step 1: Are there one or several viewpoints being expressed in the excerpt?
                    A "viewpoint" refers to "one's perspective of opinion on a particular topic".
        Step 2: If there is only one viewpoint, summarize the viewpoint in 3 keywords max,/
                    more keywords only if necessary to fully grasp the viewpoint. Viewpoint keywords/
                    should be expressed as noun phrases that describe the viewpoint in a depersonalized manner./
                    For example, instead of “Supports Renewables”, the viewpoint keyword should be “Support for Renewables”.
                    Instead of “Believes in Traditional Energy”, the viewpoint keyword should be “Belief in Traditional Energy”.

                    If there are several viewpoints, summarize each viewpoint in 3 keywords max, more keywords only if
                    necessary to fully grasp the topic. Viewpoint keywords should be expressed as noun phrases that describe
                    the viewpoint in a depersonalized manner, as explained in the instruction for one viewpoint.
        Step 3: Disregard viewpoints that are not relevant to the current topic or more relevant to another topic. Only if a viewpoint is equally relevant to multiple topics, include it under all relevant topics.
        Step 4: Identify viewpoints that convey essentially the same stance on the topic. For example, viewpoints like 'Lack of regulation and transparency' and 'Need for better regulation' express similar concerns regarding the need for increased regulation in the domain. In such cases, merge these viewpoints into a single unified viewpoint that encapsulates both perspectives. Ensure this is reflected in the summary of viewpoints in the result dictionary.

        Return the list of viewpoints in json format.


        '''{transcript}'''`
        """

prompt = PromptTemplate(template=template, input_variables=["transcript", "topics"])

viewpoints_chain = LLMChain(prompt=prompt, llm=llm)

In [ ]:
template = """
            `You are an assistant for group discussions, specialized on keeping track and documenting the discussion,
            that is, the topics discussed, the viewpoints and sub-viewpoints on each topic and the arguments/explanations given in support of each viewpoint and sub-viewpoint.
            Loop through each viewpoint and sub-viewpoint {viewpoints} and extract the arguments/explanations given in support of each viewpoint and sub-viewpoint from the corresponding excerpt in the below discussion transcript, delimited by triple hashtags.
            Your task is to identify all the arguments/explanations given in support of each viewpoint and sub-viewpoint, summarize the arguments/explanations.
            For each viewpoint, create a list of arguments in the following json format: [["viewpoint1","argument1","viewpoint1","argument2"]].


            Proceed according to the following steps:
            Step 1: For each identified viewpoint and sub-viewpoint, extract all the argument given in support of the viewpoint/sub-viewpoint from the corresponding excerpt in the below discussion transcript.
                    An "argument" refers to a statement or series of statements in support of a viewpoint expressed on a discussion topic.
                    It can consist a series of statements, facts, or any kind of explanation or justification intended to develop or support a point of view.
                    It is often structured as follows: a claim backed up with evidence, facts, and examples.

            Step 2: Summarize all the arguments per viewpoint or sub-viewpoint in one or multiple sentences.Make the summary long enough to capture the full complexity of the argument and make it understandable for an outsider unfamiliar with the discussion, but shorter than the corresponding discussion excerpt. Arguments should be expressed as noun phrases that describe the argument in a depersonalized manner.
                    For example, instead of “Argues renewables are bad, because windmills destroy biodiversity”, the argument summary should be “Renewables are bad, because wind farms negatively impact biodiversity”.


            Return the list of arguments in json format.

            ```{transcript}```
           """

prompt = PromptTemplate(template=template, input_variables=["transcript", "viewpoints"])

arguments_chain = LLMChain(prompt=prompt, llm=llm)

In [ ]:
print(viewpoints_chain.run(transcript1)) # Social Media Transcript

In [ ]:
print(arguments_chain.run(transcript1)) # Social Media Transcript